In [1]:
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.utils import extmath
from scipy.spatial.distance import pdist, squareform
from scipy import sparse, stats, spatial
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import StandardScaler

In [2]:
data_folder = './data/Main/'

# 1. Preparing the datasets

## 1.1. Loading

In [3]:
#### Loading 'tracks' dataset
tracks = pd.read_csv(data_folder+'tracks.csv', 
                        index_col=0, encoding='utf-8', low_memory=False)
# Mapping trackID to numeric ID
# Getting unique trackID
unique_trackID = tracks.trackID.unique()
# Creating a dictionary to assign numeric ID to each trackID
dict_trackID = dict(zip(unique_trackID, range(unique_trackID.size)))
# Mapping trackID to numeric ID using 'dict_trackID' dictionary
tracks['trackID'] = tracks.trackID.map(dict_trackID)

print(tracks.shape)
tracks.head(3)

(64500, 14)


,trackID,trackTitle,artistName,trackPopularity,releaseDate,artistFollowers,topGenre1,topGenre2,artistID,albumID,songID,audio_Duration_ms,set_subset,set_split
0,0,The Room Of Stairs,Harold Budd,11.0,2000,30414.0,Electronic/Dance,Classical,3uOCouLFR4bVx0XeiQJSbl,3jq7b66l8MswqDmi0mxzjq,SOAAADD12AB018A9DD,321133,medium,train
1,1,(I'm Gonna Start) Living Again If It Kills Me,Dave Edmunds,5.0,1981,28604.0,Rock,Blues,65Gh3BfK84aTIugiRCgLBA,0upTl2RUS4gmStWBlXjt9l,SOAAADE12A6D4F80CC,199088,medium,train
2,2,KRYSAR (LIVE),LANDA DANIEL,15.0,2003,NaN,NaN,NaN,6rSpV5hdCNJ4v1i602nj22,7uvLu2RAki7aKiASSxJPzi,SOAAADF12A8C13DF62,200880,medium,train


In [4]:
### Loading playlists dataset
user_profile = pd.read_csv(data_folder+'userProfile.csv', 
                        index_col=0, encoding='utf-8', low_memory=False)

# Keeping only tracks that exists in tracks dataframe
user_profile = user_profile[user_profile.trackID.isin(list(dict_trackID.keys()))]

print(user_profile.shape)
user_profile.head(3)

/root/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4911782, 4)


,userID,songID,counts,trackID
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2,2ECKXkpPAxky87ohawpaeD
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBVFZR12A6D4F8AE3,1,5WjXULJvSlMxuies9diz1Q
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXALG12A8C13C108,1,1bqi9YEdZweX9H6JuSQ6Qw


In [5]:
### Mapping userID to numeric ID
# getting unique userID
unique_userID = user_profile.userID.unique()
# Creating a dictionary to assign numeric ID to current user ID
dict_userID = dict(zip(unique_userID, range(unique_userID.size)))
# Mapping userID using prepared dictionary
user_profile.userID = user_profile.userID.map(dict_userID)
# Mapping trackID to numeric ID using 'dict_trackID' dictionary
user_profile.trackID = user_profile.trackID.map(dict_trackID)
user_profile = user_profile[['userID', 'trackID', 'counts']]

print(user_profile.shape)

(4911782, 3)


In [6]:
# Loading 'librosa_features' dataset
features = pd.read_csv(data_folder+'librosa_features.csv', 
                        index_col=0, header=[0,1,2], encoding='utf-8', low_memory=False)

# Converting multi-index columns to flat index
features.columns = [ '_'.join(x) for x in features.columns ]

# Filter Librosa feature to the songs that exists in tracks dataframe
features = features[features.index.isin(list(dict_trackID.keys()))]

# Mapping trackID to numeric ID using 'dict_trackID' dictionary
features.index = features.index.map(dict_trackID).astype(int)

print(features.shape)
features.head(3)

(64500, 518)


,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,chroma_cens_kurtosis_10,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,0.720673,4.240027,-0.618995,0.426698,-0.226265,-0.209698,-0.317341,-1.345874,-0.586116,-0.931247,...,0.084099,0.017905,0.017880,3.022421,0.400879,0.093924,0.078125,0.030273,1.610670,0.054095
1,-0.370662,-0.287299,1.182948,-0.952800,-0.790715,-0.504890,-0.491160,0.696574,0.054538,0.780068,...,0.127805,0.020186,0.023055,13.629067,0.286621,0.046816,0.042969,0.008301,2.550661,0.024498
2,-0.769198,-0.571088,-0.860308,0.437645,-1.027899,1.226321,-0.928717,1.680385,-1.276514,-0.994085,...,0.084305,0.018808,0.020520,10.252457,0.222168,0.053373,0.049316,0.013184,2.536385,0.023678


## 1.2. Filtering to 12,000 songs

In [7]:
# Filtering tracks to be left by only small subset
tracks_s = tracks[tracks.set_subset=='small']#.iloc[:2000,:]
print('Tracks dataframe shape after filtering: ',tracks_s.shape)

# Filtering features based on filtered 'tracks_s' dataframe
features_s  = features[features.index.isin(tracks_s.trackID)]
# Filling NAs
features_s = features_s.fillna(0)
print('Features dataframe after ',features_s.shape)

Tracks dataframe shape after filtering:  (12900, 14)
Features dataframe after  (12900, 518)


# 2. Songs similarity network based on Librosa Features

## 2.1. Sparsifying the network

In [8]:
#######################################################################################################
###### Standardizing features, since the algorithm works better with standardized features #######
#######################################################################################################

# Standardizing the 'features' dataframe
scaler_features = StandardScaler()
features_scaled = scaler_features.fit_transform(features_s)
features_scaled = pd.DataFrame(features_scaled, index=features_s.index, columns=features_s.columns)

# Compute the distance between the tracks for creating adjacency matrix
### Using pdist to calculate the pairwise distance between each track
distances = spatial.distance.pdist(features_scaled, metric='cosine')
distances = spatial.distance.squareform(distances)

print(distances.shape)

(12900, 12900)


In [9]:
# Computing weights matrix
kernel_width = distances.mean()
weights = np.exp(-1*(distances**2/(kernel_width**2)))
#Setting the diagonal to zero
np.fill_diagonal(weights, 0)

# First step of sparsifying: Based on weights
sp_thr = 0.1
weights[weights<=sp_thr] = 0

# We sparsify based on outdegree of each node: a[0,:] show out-going edges
### This is because in the algorithms that we will use, 
### ...in-degree contains information that we don't want to lose
Edges = 30
sort_idx = np.argsort(weights, axis=1)
#Leaving all but the 50 strongest edges
for i,index in enumerate(sort_idx):
    weights[i, index[:-Edges]] = 0

## 2.2. Creating NetworkX graph

In [10]:
### Getting sparse matrix
weights_sp = sparse.csr_matrix(weights)
# Building a graph from Sparse CSR matrix
G_tracks = nx.from_scipy_sparse_matrix(weights_sp, create_using=nx.DiGraph(), edge_attribute='weight')
# Relabeling the nodes (name of nodes: TrackID)
G_tracks = nx.relabel_nodes(G_tracks, dict(zip(range(len(tracks_s.trackID.values)), tracks_s.trackID.values)))

### Getting connected components
components = []
for component in nx.strongly_connected_components(G_tracks):
    components.append(component)
# Sorting the connected components based on their number of nodes
components.sort(key=lambda x: len(x))
# Creating a subgraph of G_tracks whihc contains selected songs 
### (here 500 songs selected in "creating utility matrix" step)
G_tracks = G_tracks.subgraph(sorted(list(components[-1])))

# Adding meta data to the graph
for node, dic in G_tracks.nodes(data=True):
    G_tracks.node[node]['trackTitle'] = tracks.loc[tracks.trackID==node,'trackTitle'].values[0]
    G_tracks.node[node]['artistName'] = tracks.loc[tracks.trackID==node,'artistName'].values[0]
    G_tracks.node[node]['topGenre'] = tracks.loc[tracks.trackID==node,'topGenre1'].values[0]
    G_tracks.node[node]['audioDuration'] = tracks.loc[tracks.trackID==node,'audio_Duration_ms'].values[0]//1000
    G_tracks.node[node]['trackPopularity'] = tracks.loc[tracks.trackID==node,'trackPopularity'].values[0]

In [11]:
# Creating a list of nodes names
nodes_name = tracks_s.trackID.values 
nodes_name = nodes_name[np.isin(nodes_name, list(G_tracks.nodes()))]

## 2.3. Creating output files for Gephi

In [12]:
nodes_id_gephi = []
nodes_labels_gephi = []
for i, node in enumerate(G_tracks.nodes(data=True)):
    nodes_labels_gephi.append(node[1]['artistName'])
    nodes_id_gephi.append(node[0])
# Creating the dataframe    
gephi_nodes_df = pd.DataFrame(dict(node=nodes_id_gephi, label=nodes_labels_gephi))
# Saving the nodes dataframe
#gephi_nodes_df.to_csv(data_folder+'gephi_nodes.csv', sep=';', encoding='utf-8', index=False)

gephi_nodes_df.head()

,node,label
0,32772,Adam Arcuragi
1,5,Ingmar Nordströms
2,10,The Wolfgang Press
3,11,Bobby Pullido
4,32786,Agnostic Front


In [13]:
source_id_gephi = []
target_id_gephi = []
weights_gephi = []
for i, edge in enumerate(G_tracks.edges(data=True)):
    source_id_gephi.append(edge[0])
    target_id_gephi.append(edge[1])
    weights_gephi.append(edge[2]['weight'])
# Creating the dataframe   
gephi_edges_df = pd.DataFrame(dict(source=source_id_gephi, target=target_id_gephi, 
                                   weight=weights_gephi))
# Saving the nodes dataframe
#gephi_edges_df.to_csv(data_folder+'gephi_edges.csv', encoding='utf-8', index=False)

gephi_edges_df.head()

,source,target,weight
0,32772,3918,0.897446
1,32772,8181,0.898069
2,32772,8522,0.882573
3,32772,10249,0.901878
4,32772,10549,0.880082


# 3. Songs similarity network extracted from User profile

## 3.1. Creating the Utility matrix

In [14]:
# Loading the saved mapped 'user_profile'
user_profile_s = pd.read_csv(data_folder+'user_profile_mapped_s.csv', encoding='utf-8', index_col=0)

/root/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
# Creating a sparse matrix from 'user_profile' dataframe
userID_cat = CategoricalDtype(user_profile_s.userID.unique(), ordered=True)
trackID_cat = CategoricalDtype(nodes_name, ordered=True)

row = user_profile_s.userID.astype(userID_cat).cat.codes
col = user_profile_s.trackID.astype(trackID_cat).cat.codes
sparse_utility = csr_matrix((user_profile_s['counts'], (row, col)), \
                             shape=(userID_cat.categories.size, trackID_cat.categories.size))

# Defining the sparse DataFrame
utility_df = pd.SparseDataFrame(sparse_utility, \
                         index=userID_cat.categories, \
                         columns=trackID_cat.categories, \
                         default_fill_value=0)


## 3.1. Decomposing the User profile dataframe

In [16]:

n_components = 50
U, S, V = extmath.randomized_svd(sparse_utility, n_components, n_oversamples=20, n_iter=10, 
                                               power_iteration_normalizer='QR', 
                                               transpose=False, flip_sign=True, random_state=0)

In [17]:
# Compute the distance between the tracks for creating adjacency matrix
### Using pdist to calculate the pairwise distance between each track
distances_h = spatial.distance.pdist(V.T, metric='cosine')
distances_h = spatial.distance.squareform(distances_h)

# Computing weights matrix
kernel_width_h = distances_h.mean()
weights_h = np.exp(-1*(distances_h**2/(kernel_width_h**2)))
#Setting the diagonal to zero
np.fill_diagonal(weights_h, 0)

# First step of sparsifying: Based on weights
sp_thr = 0.2
weights_h[weights_h<=sp_thr] = 0

# We sparsify based on outdegree of each node: a[0,:] show out-going edges
### This is because in the algorithms that we will use, 
### ...in-degree contains information that we don't want to lose
Edges = 30

sort_idx = np.argsort(weights_h, axis=1)
#Leaving all but the 50 strongest edges
for i,index in enumerate(sort_idx):
    weights_h[i, index[:-Edges]] = 0

## 3.2. Creating the NetworkX graph

In [18]:
# Getting sparse matrix
weights_sp = sparse.csr_matrix(weights_h)
# Building a graph from Sparse CSR matrix
G_tracks_h = nx.from_scipy_sparse_matrix(weights_sp, create_using=nx.DiGraph(), edge_attribute='weight')
# Relabeling the nodes (name of nodes: TrackID)
G_tracks_h = nx.relabel_nodes(G_tracks_h, dict(zip(range(len(utility_df.columns.values)), utility_df.columns.values)))

# Adding meta data to the graph
for node, dic in G_tracks_h.nodes(data=True):
    G_tracks_h.node[node]['trackTitle'] = tracks.loc[tracks.trackID==node,'trackTitle'].values[0]
    G_tracks_h.node[node]['artistName'] = tracks.loc[tracks.trackID==node,'artistName'].values[0]
    G_tracks_h.node[node]['topGenre'] = tracks.loc[tracks.trackID==node,'topGenre1'].values[0]
    G_tracks_h.node[node]['audioDuration'] = tracks.loc[tracks.trackID==node,'audio_Duration_ms'].values[0]//1000
    G_tracks_h.node[node]['trackPopularity'] = tracks.loc[tracks.trackID==node,'trackPopularity'].values[0]

## 3.3. Creating output files for Gephi

In [19]:
nodes_id_gephi = []
nodes_labels_gephi = []
for i, node in enumerate(G_tracks_h.nodes(data=True)):
    nodes_labels_gephi.append(node[1]['artistName'])
    nodes_id_gephi.append(node[0])
# Creating the dataframe 
gephi_nodes_df_h = pd.DataFrame(dict(node=nodes_id_gephi, label=nodes_labels_gephi))
# Saving the nodes dataframe
#gephi_nodes_df_h.to_csv(data_folder+'gephi_nodes_p.csv', sep=';', encoding='utf-8', index=False)

gephi_nodes_df_h.head()

,node,label
0,5,Ingmar Nordströms
1,10,The Wolfgang Press
2,11,Bobby Pullido
3,21,London Elektricity
4,22,Blitzen Trapper


In [20]:
source_id_gephi = []
target_id_gephi = []
weights_gephi = []

for i, edge in enumerate(G_tracks_h.edges(data=True)):
    source_id_gephi.append(edge[0])
    target_id_gephi.append(edge[1])
    weights_gephi.append(edge[2]['weight'])
# Creating the the dataframe
gephi_edges_df_h = pd.DataFrame(dict(source=source_id_gephi, target=target_id_gephi, 
                                   weight=weights_gephi))
# Saving the nodes dataframe
#gephi_edges_df_h.to_csv(data_folder+'gephi_edges_p.csv', encoding='utf-8', index=False)

gephi_edges_df_h.head()

,source,target,weight
0,5,1000,0.991048
1,5,6625,0.989474
2,5,10272,0.994391
3,5,10742,0.993712
4,5,11411,0.990570
